In [75]:
f = open("CLIENTS.csv")
s = f.read()
s = s.replace(";.", ";")
g = open("NEW_CLIENTS.csv", "w")
g.write(s)
g.close()

In [76]:
import pandas as pd
import numpy as np

X = pd.read_csv("NEW_CLIENTS.csv", sep=";")
X = X[X["PERF_class"] != "EXCL"]
y = X["PERF_class"].map(dict(GOOD=1, BAD=0))
X = X.drop(["ID", "PERF_class", "PERF_situation"], axis = 1)
to_drop = []

for i in range(len(X.columns)):
    if ((X.iloc[:, i].count() - list(X.iloc[:, i]).count(".")) / float(len(X))) < 0.9:
        to_drop.append(i)


print to_drop

for i in reversed(to_drop):
    X = X.drop(X.columns[i], axis = 1)


for i in range(len(X.columns)):
    if np.issubdtype(X.iloc[:, i].dtype, np.number):
        X.iloc[:,i] = X.iloc[:, i].fillna(X.iloc[:, i].mean())
    else:
        X.iloc[:,i] = X.iloc[:, i].fillna(X.iloc[:, i].value_counts().index[0])

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

for column in X.columns:
    if ( not np.issubdtype(X.loc[:, column].dtype, np.number)):
        lb_results = lb.fit_transform(X.loc[:, column])
        for i in range(len(lb_results[0])):
            X.insert(len(X.columns), column + str(i), lb_results[:, i])
        X = X.drop(column, axis = 1)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X[X.columns])
X.head()

[1, 3, 6, 8, 28, 29, 40]


,CLIENT_BDFCLI,CLIENT_CSP,CLIENT_NBENF,CLIENT_age,CLIENT_ancembauche,CLIENT_anchabitat,CLIENT_endettement,CLIENT_loyer,CLIENT_revenus,CREDIT_agios,...,VEHICULE_type126,VEHICULE_type127,VEHICULE_type128,VEHICULE_type129,VEHICULE_type130,VEHICULE_type131,VEHICULE_type132,VEHICULE_type133,VEHICULE_type134,VEHICULE_type135
1,-5.774718,-0.056454,0.347569,-0.015478,-0.953637,0.699447,-0.259369,-0.731516,0.125740,-0.516125,...,-0.094665,-0.163903,-0.040884,-0.007337,-0.165626,-0.010376,-0.036709,-0.050897,-0.014675,-0.123481
2,0.117975,-1.657584,0.347569,0.700441,1.964893,1.794420,-0.259369,3.798143,13.261917,-0.552891,...,-0.094665,-0.163903,-0.040884,-0.007337,-0.165626,-0.010376,-0.036709,-0.050897,-0.014675,-0.123481
3,0.117975,-1.324015,2.394062,0.342481,-0.586636,0.240018,-0.259369,-0.731516,2.969675,-0.388248,...,-0.094665,-0.163903,-0.040884,-0.007337,-0.165626,-0.010376,-0.036709,-0.050897,-0.014675,-0.123481
4,0.117975,0.944253,-0.675678,1.631135,-0.088564,-0.464440,-0.259369,1.148293,0.333335,-0.551355,...,-0.094665,-0.163903,-0.040884,-0.007337,-0.165626,-0.010376,-0.036709,-0.050897,-0.014675,-0.123481
5,0.117975,-0.256595,-0.675678,0.199298,2.034798,-0.786040,-0.259369,0.400899,-0.427585,-0.357261,...,-0.094665,-0.163903,-0.040884,-0.007337,-0.165626,-0.010376,-0.036709,-0.050897,-0.014675,-0.123481


In [77]:
from sklearn.linear_model import LogisticRegression

X1 = X.iloc[:10000, :]
X2 = X.iloc[10000:, :]
y1 = y.iloc[:10000]
y2 = y.iloc[10000:]

my_model = LogisticRegression(class_weight='balanced')
my_model.fit(X1, y1)

#print(my_model.coef_[0])
#print(my_model.intercept_[0])

prediction = my_model.predict( X2 )
prediction_proba = my_model.predict_proba(X2)

"""
error = [0, 0]
total = [0, 0]
for i in range(len(X)):
    if y.iloc[i] == "GOOD":
        total[0] += 1
        if prediction[i] == "BAD":
            error[0] += 1
    else:
        total[1] += 1
        if prediction[i] == "GOOD":
            error[1] += 1

print(error)
print(total)
"""
print(prediction_proba)


#predict_proba = [1] * len(y)
# ROC = 1 - perfect classifier
# naive classifier : all GOOD or all BAD: ROC = 0.5

[[  1.10011787e-05   9.99988999e-01]
 [  9.95186901e-05   9.99900481e-01]
 [  2.35262173e-05   9.99976474e-01]
 ..., 
 [  1.59782742e-04   9.99840217e-01]
 [  2.46529688e-03   9.97534703e-01]
 [  8.91389042e-04   9.99108611e-01]]


In [78]:
from sklearn.metrics import roc_auc_score
naive_prediction = [1] * len(y2)
roc_auc_score(y2, prediction_proba[:, 1]) 

0.71177584816320372

In [79]:
A = sorted(zip(X.columns, my_model.coef_[0]), key = lambda x: -abs(x[1]))
important_columns = [x[0] for x in A] 
important_columns

['CREDIT_agios',
 'CREDIT_montant',
 'CREDIT_apport',
 'CREDIT_duree',
 'CLIENT_revenus',
 'CREDIT_date1er',
 'VEHICULE_type103',
 'CREDIT_fraisdossier',
 'CREDIT_saisie215',
 'CREDIT_taeg',
 'CREDIT_saisie246',
 'CLIENT_ancembauche',
 'VEHICULE_type19',
 'VEHICULE_type53',
 'VEHICULE_type20',
 'CREDIT_saisie279',
 'VEHICULE_type105',
 'CREDIT_saisie147',
 'VEHICULE_type133',
 'CREDIT_saisie216',
 'VEHICULE_type96',
 'VEHICULE_cvfisc',
 'CLIENT_situation_familiale2',
 'CREDIT_saisie267',
 'CREDIT_saisie180',
 'CLIENT_situation_familiale0',
 'VEHICULE_type116',
 'CREDIT_saisie217',
 'VEHICULE_type67',
 'CREDIT_saisie269',
 'VEHICULE_type41',
 'FIN_cout_risque_estim',
 'CREDIT_saisie192',
 'VEHICULE_type29',
 'CLIENT_habitat0',
 'CREDIT_saisie274',
 'CREDIT_saisie134',
 'VEHICULE_type9',
 'VEHICULE_type126',
 'CREDIT_saisie251',
 'CREDIT_saisie189',
 'FIN_taux_refinancement_estim',
 'CREDIT_saisie208',
 'VEHICULE_anciennete',
 'VEHICULE_type74',
 'CREDIT_saisie194',
 'VEHICULE_type76',
 

In [80]:
from sklearn.linear_model import LogisticRegression

X = X.loc[:, important_columns[:40]]

X.head()

,CREDIT_agios,CREDIT_montant,CREDIT_apport,CREDIT_duree,CLIENT_revenus,CREDIT_date1er,VEHICULE_type103,CREDIT_fraisdossier,CREDIT_saisie215,CREDIT_taeg,...,VEHICULE_type41,FIN_cout_risque_estim,CREDIT_saisie192,VEHICULE_type29,CLIENT_habitat0,CREDIT_saisie274,CREDIT_saisie134,VEHICULE_type9,VEHICULE_type126,CREDIT_saisie251
1,-0.516125,-0.445122,-0.479436,-1.172886,0.125740,-1.691478,-0.080632,-0.656053,-0.088695,-1.083591,...,-0.050363,-0.166638,-0.100293,-0.130484,-0.049277,-0.067798,-0.061501,-0.147964,-0.094665,-0.075394
2,-0.552891,0.385825,-0.096058,-1.172886,13.261917,-1.691478,-0.080632,0.105568,-0.088695,-2.036378,...,-0.050363,-0.253669,-0.100293,-0.130484,-0.049277,-0.067798,-0.061501,-0.147964,-0.094665,-0.075394
3,-0.388248,-0.200640,1.253799,-0.689541,2.969675,-1.691478,-0.080632,-0.065226,-0.088695,-0.165530,...,-0.050363,-0.333684,-0.100293,-0.130484,-0.049277,-0.067798,-0.061501,-0.147964,-0.094665,-0.075394
4,-0.551355,0.752365,-0.167104,-1.172886,0.333335,-1.691478,-0.080632,0.430891,-0.088695,-2.036378,...,-0.050363,-0.552848,-0.100293,-0.130484,-0.049277,-0.067798,-0.061501,-0.147964,-0.094665,-0.075394
5,-0.357261,-0.347347,0.099404,-0.280557,-0.427585,-1.691478,-0.080632,-0.260542,-0.088695,-0.106837,...,-0.050363,-0.336873,-0.100293,-0.130484,-0.049277,-0.067798,-0.061501,-0.147964,-0.094665,-0.075394


In [81]:
X1 = X.iloc[:10000, :]
X2 = X.iloc[10000:, ]
y1 = y.iloc[:10000]
y2 = y.iloc[10000:]

my_model = LogisticRegression(class_weight='balanced')
my_model.fit(X1, y1)

#print(my_model.coef_[0])
#print(my_model.intercept_[0])

prediction = my_model.predict( X_new )
prediction_proba = my_model.predict_proba(X2)

"""
error = [0, 0]
total = [0, 0]
for i in range(len(X)):
    if y.iloc[i] == "GOOD":
        total[0] += 1
        if prediction[i] == "BAD":
            error[0] += 1
    else:
        total[1] += 1
        if prediction[i] == "GOOD":
            error[1] += 1

print(error)
print(total)
"""
print(prediction_proba)

[[  4.06503558e-04   9.99593496e-01]
 [  8.17879174e-04   9.99182121e-01]
 [  1.65514646e-04   9.99834485e-01]
 ..., 
 [  5.74948512e-02   9.42505149e-01]
 [  1.71373106e-01   8.28626894e-01]
 [  6.69557479e-02   9.33044252e-01]]


In [82]:
from sklearn.metrics import roc_auc_score
naive_prediction = [1] * len(y2)
roc_auc_score(y2, prediction_proba[:, 1]) 

0.76228123433859718

In [83]:
import datetime
A = datetime.date(1991, 1, 1)
B = datetime.date(2018, 1, 1)

print((B-A).days)

9862


In [84]:
X = pd.read_csv("NEW_CLIENTS.csv", sep=";")

X["CREDIT_saisie"] = [(datetime.date(int(S[0:4]), int(S[5:7]), int(S[8:10])) - datetime.date(2015, 1, 1)).days for S in X["CREDIT_saisie"]]
X["CREDIT_saisie"]

0       -167
1       -116
2        -82
3        -78
4        -71
5        -64
6        -59
7        -55
8        -50
9        -48
10       -48
11       -47
12       -44
13       -43
14       -41
15       -40
16       -38
17       -37
18       -37
19       -36
20       -33
21       -33
22       -33
23       -30
24       -29
25       -29
26       -28
27       -28
28       -28
29       -28
        ... 
20124    167
20125    167
20126    167
20127    167
20128    167
20129    167
20130    167
20131    167
20132    167
20133    167
20134    167
20135    167
20136    167
20137    167
20138    167
20139    167
20140    167
20141    167
20142    167
20143    167
20144    167
20145    167
20146    167
20147    167
20148    167
20149    167
20150    167
20151    167
20152    167
20153    167
Name: CREDIT_saisie, dtype: int64